In [105]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotting import (
    plot_merchant_type_distribution,
    plot_transaction_amount_distribution,
    plot_event_distribution,
    plot_behavior_over_time,
    display_table_info
)

In [106]:
# Load data
account_state_log = pd.read_csv('Datasets/account_state_log.csv')
transactions_log = pd.read_csv('Datasets/transactions_log.csv', parse_dates=['timestamp'])
payments_log = pd.read_csv('Datasets/payments_log.csv', parse_dates=['timestamp'])
labels=pd.read_csv('Datasets/labels.csv')

In [107]:
account_state_log

,agent_id,credit_balance,credit_utilization,interest_rate,min_payment_factor,current_missed_payments,timestamp
0,839,367.017294,0.367,0.191914,0.05,0,2023-01-01
1,179,638.371473,0.638,0.191758,0.05,0,2023-01-01
2,645,652.219191,0.326,0.235719,0.05,0,2023-01-01
3,170,777.674485,0.078,0.234455,0.05,0,2023-01-01
4,152,634.101608,0.317,0.247455,0.05,0,2023-01-01
...,...,...,...,...,...,...,...
373220,16481,2948.692830,0.983,0.196236,0.05,0,2023-04-01
373221,10703,653.288549,0.131,0.232751,0.05,0,2023-04-01
373222,13290,4700.421460,0.470,0.220961,0.05,0,2023-04-01
373223,17809,1055.419737,0.211,0.238539,0.05,0,2023-04-01


In [108]:
len(account_state_log['agent_id'].unique())

7352

In [109]:
aggregated = account_state_log.groupby('agent_id').agg({
    'credit_balance': ['mean', 'max', 'last'],  # Average, peak, and final balance
    'credit_utilization': ['mean', 'max', 'last'],  # Utilization stats
    'interest_rate': ['mean', 'last'],  # Average and final interest rate
    'current_missed_payments': ['sum', 'max', 'last'],  # Total, peak, and latest missed payments
    'timestamp': ['min', 'max']  # First and last timestamps
})

# Flatten MultiIndex columns for easier use
aggregated.columns = ['_'.join(col).strip() for col in aggregated.columns]
aggregated.reset_index(inplace=True)

In [110]:
aggregated

,agent_id,credit_balance_mean,credit_balance_max,credit_balance_last,credit_utilization_mean,credit_utilization_max,credit_utilization_last,interest_rate_mean,interest_rate_last,current_missed_payments_sum,current_missed_payments_max,current_missed_payments_last,timestamp_min,timestamp_max
0,0,3054.533833,4964.111415,2505.991415,0.623747,0.993,0.501,0.209866,0.209866,0,0,0,2023-01-01,2023-04-01
1,3,1826.206224,1990.626004,1990.626004,0.927758,0.995,0.995,0.211185,0.211185,0,0,0,2023-01-01,2023-04-01
2,6,649.106037,1279.993400,321.083400,0.227275,0.439,0.107,0.186929,0.187204,0,0,0,2023-01-01,2023-04-01
3,7,1621.523367,1984.664026,1941.344026,0.850923,0.998,0.971,0.216252,0.216252,0,0,0,2023-01-01,2023-04-01
4,8,5748.472329,9973.203428,7835.223428,0.584846,0.997,0.784,0.219701,0.219975,1,1,0,2023-01-01,2023-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,18185,691.140065,691.140065,691.140065,0.346000,0.346,0.346,0.153258,0.153258,0,0,0,2023-04-01,2023-04-01
7348,18186,1167.109351,1167.109351,1167.109351,0.389000,0.389,0.389,0.206696,0.206696,0,0,0,2023-04-01,2023-04-01
7349,18187,6.503746,6.503746,6.503746,0.145000,0.145,0.145,0.181687,0.181687,0,0,0,2023-04-01,2023-04-01
7350,18189,575.152999,575.152999,575.152999,0.288000,0.288,0.288,0.217025,0.217025,0,0,0,2023-04-01,2023-04-01


In [111]:
transactions_log

,agent_id,status,amount,merchant_category,merchant_id,online,timestamp
0,839,approved,8.64,Misc,EPJJ1K11,0,2023-01-01 02:13:35
1,179,approved,114.95,Misc,90GGX8J3,1,2023-01-01 01:38:41
2,645,approved,160.78,Retail,D33HTW5R,0,2023-01-01 07:29:55
3,170,approved,281.63,Business,FZR3PHV3,0,2023-01-01 05:46:06
4,152,approved,39.13,Clothing,4Z8KX9G4,0,2023-01-01 10:40:22
...,...,...,...,...,...,...,...
461699,10224,declined,39.13,Clothing,DBY4WEHR,1,2023-03-31 02:46:31
461700,13972,approved,97.62,Retail,D33HTW5R,1,2023-03-31 05:00:19
461701,484,approved,97.62,Retail,PI2C86AP,1,2023-03-31 03:59:00
461702,10350,approved,89.15,Transportation,U7FJ5ZPN,0,2023-03-31 05:54:04


In [112]:
len(transactions_log['agent_id'].unique())

7275

In [113]:
# Group by `agent_id` and aggregate
aggregated_transactions = transactions_log.groupby('agent_id').agg({
    'amount': ['sum', 'mean', 'max', 'count'],  # Transaction value stats
    'status': lambda x: (x == 'completed').sum(),  # Count of completed transactions
    'merchant_category': lambda x: x.mode()[0] if not x.mode().empty else None,  # Most frequent category
    'merchant_id': pd.Series.nunique,  # Unique merchants
    'online': ['sum', 'mean'],  # Total and proportion of online transactions
    'timestamp': ['min', 'max']  # Transaction timeline
})

# Flatten MultiIndex columns
aggregated_transactions.columns = ['_'.join(col).strip() for col in aggregated_transactions.columns]
aggregated_transactions.reset_index(inplace=True)

# Rename for clarity
aggregated_transactions.rename(columns={
    'amount_sum': 'total_transaction_amount',
    'amount_mean': 'average_transaction_amount',
    'amount_max': 'max_transaction_amount',
    'amount_count': 'total_transactions',
    'status_<lambda>': 'completed_transactions',
    'merchant_id_nunique': 'unique_merchants',
    'online_sum': 'online_transactions',
    'online_mean': 'online_transaction_proportion',
    'timestamp_min': 'first_transaction_date',
    'timestamp_max': 'last_transaction_date'
}, inplace=True)

aggregated_transactions

,agent_id,total_transaction_amount,average_transaction_amount,max_transaction_amount,total_transactions,completed_transactions,merchant_category_<lambda>,unique_merchants,online_transactions,online_transaction_proportion,first_transaction_date,last_transaction_date
0,0,10812.48,124.281379,289.45,87,0,Misc,46,35,0.402299,2023-01-01 05:30:35,2023-03-31 08:16:57
1,3,47134.16,228.806602,591.81,206,0,Business,62,88,0.427184,2023-01-01 06:22:40,2023-03-31 04:20:35
2,6,2748.74,30.541556,107.45,90,0,Misc,31,47,0.522222,2023-01-01 05:30:59,2023-03-31 07:03:23
3,7,36366.21,227.288813,591.81,160,0,Misc,61,77,0.481250,2023-01-01 06:08:52,2023-03-31 08:46:48
4,8,27102.78,218.570806,591.81,124,0,Misc,52,55,0.443548,2023-01-01 04:41:12,2023-03-31 03:29:17
...,...,...,...,...,...,...,...,...,...,...,...,...
7270,18094,8.64,8.640000,8.64,1,0,Misc,1,1,1.000000,2023-03-31 03:30:38,2023-03-31 03:30:38
7271,18095,281.63,281.630000,281.63,1,0,Business,1,1,1.000000,2023-03-31 10:12:00,2023-03-31 10:12:00
7272,18097,281.63,281.630000,281.63,1,0,Business,1,0,0.000000,2023-03-31 03:16:59,2023-03-31 03:16:59
7273,18098,281.63,281.630000,281.63,1,0,Business,1,1,1.000000,2023-03-31 03:07:24,2023-03-31 03:07:24


In [114]:
transactions_log = pd.get_dummies(transactions_log, columns=['merchant_category'], prefix='', prefix_sep='')

# Now, perform your aggregation as before
unique_categories = ['Misc', 'Retail', 'Business', 'Clothing', 'Agricultural', 
                        'Contractor', 'Transportation', 'Utility', 'Professional']

# Group by `agent_id` and aggregate other columns
aggregated_transactions = transactions_log.groupby('agent_id').agg({
    'amount': ['sum', 'mean', 'max', 'count'],  # Transaction value stats
    'online': ['sum', 'mean'],  # Total and proportion of online transactions
    'timestamp': ['min', 'max'],  # Transaction timeline
}).reset_index()

# Aggregate the merchant category counts (sum the binary columns)
for category in unique_categories:
    aggregated_transactions[category] = transactions_log.groupby('agent_id')[category].sum().reset_index(drop=True)

# Flatten multi-level columns in the aggregated DataFrame
aggregated_transactions.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in aggregated_transactions.columns]

# Add custom metrics for `status` outside of the main aggregation
status_counts = transactions_log.groupby('agent_id')['status'].value_counts().unstack(fill_value=0).reset_index()
status_counts.rename(columns={'approved': 'approved_count', 'declined': 'declined_count'}, inplace=True)

# Merge the status counts back with the aggregated data
aggregated_transactions = pd.merge(aggregated_transactions, status_counts, on='agent_id', how='left')

# Drop the `merchant_id` column if it exists (optional, as per your requirement)
aggregated_transactions = aggregated_transactions.drop(columns=[col for col in aggregated_transactions.columns if 'merchant_id' in col], errors='ignore')

# Display the final aggregated transactions DataFrame
aggregated_transactions

,agent_id,amount_sum,amount_mean,amount_max,amount_count,online_sum,online_mean,timestamp_min,timestamp_max,Misc,Retail,Business,Clothing,Agricultural,Contractor,Transportation,Utility,Professional,approved_count,declined_count
0,0,10812.48,124.281379,289.45,87,35,0.402299,2023-01-01 05:30:35,2023-03-31 08:16:57,19,7,5,8,8,7,15,11,7,66,21
1,3,47134.16,228.806602,591.81,206,88,0.427184,2023-01-01 06:22:40,2023-03-31 04:20:35,52,27,53,12,20,5,11,10,16,23,183
2,6,2748.74,30.541556,107.45,90,47,0.522222,2023-01-01 05:30:59,2023-03-31 07:03:23,60,8,0,7,4,0,5,2,4,90,0
3,7,36366.21,227.288813,591.81,160,77,0.481250,2023-01-01 06:08:52,2023-03-31 08:46:48,41,17,36,14,16,10,8,10,8,48,112
4,8,27102.78,218.570806,591.81,124,55,0.443548,2023-01-01 04:41:12,2023-03-31 03:29:17,38,9,34,11,8,3,9,6,6,75,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7270,18094,8.64,8.640000,8.64,1,1,1.000000,2023-03-31 03:30:38,2023-03-31 03:30:38,1,0,0,0,0,0,0,0,0,1,0
7271,18095,281.63,281.630000,281.63,1,1,1.000000,2023-03-31 10:12:00,2023-03-31 10:12:00,0,0,1,0,0,0,0,0,0,1,0
7272,18097,281.63,281.630000,281.63,1,0,0.000000,2023-03-31 03:16:59,2023-03-31 03:16:59,0,0,1,0,0,0,0,0,0,1,0
7273,18098,281.63,281.630000,281.63,1,1,1.000000,2023-03-31 03:07:24,2023-03-31 03:07:24,0,0,1,0,0,0,0,0,0,1,0


In [115]:
payments_log

,agent_id,amount,timestamp
0,881,322.69,2023-01-01 05:28:17
1,331,483.74,2023-01-01 08:16:48
2,390,292.20,2023-01-01 04:08:06
3,939,337.62,2023-01-01 06:48:56
4,120,614.31,2023-01-01 04:33:32
...,...,...,...
20386,15603,521.89,2023-03-31 07:04:09
20387,16960,1232.65,2023-03-31 03:54:48
20388,16526,413.18,2023-03-31 02:30:08
20389,13809,1170.80,2023-03-31 09:08:47


In [116]:
# Group by `agent_id` and aggregate
aggregated_payments = payments_log.groupby('agent_id').agg({
    'amount': ['sum', 'mean', 'max', 'min', 'count', 'last'],  # Payment stats
    'timestamp': ['min', 'max']  # Payment timeline
})

# Flatten MultiIndex columns
aggregated_payments.columns = ['_'.join(col).strip() for col in aggregated_payments.columns]
aggregated_payments.reset_index(inplace=True)

# Add derived features (e.g., payment frequency, time difference)
aggregated_payments['payment_frequency'] = aggregated_payments['amount_count'] / (
    (aggregated_payments['timestamp_max'] - aggregated_payments['timestamp_min']).dt.days + 1
)  # Payments per day
aggregated_payments['time_between_first_last'] = (
    aggregated_payments['timestamp_max'] - aggregated_payments['timestamp_min']
).dt.days

In [117]:
aggregated_payments

,agent_id,amount_sum,amount_mean,amount_max,amount_min,amount_count,amount_last,timestamp_min,timestamp_max,payment_frequency,time_between_first_last
0,0,5878.93,2939.465000,2953.11,2925.82,2,2953.11,2023-02-20 09:05:01,2023-03-27 06:20:56,0.057143,34
1,3,2670.97,890.323333,925.06,848.27,3,925.06,2023-01-16 06:47:01,2023-03-02 03:30:47,0.066667,44
2,6,2965.40,370.675000,612.19,227.14,8,264.32,2023-01-19 04:47:47,2023-03-30 06:09:23,0.112676,70
3,7,7318.15,1045.450000,1187.64,427.18,7,1187.64,2023-01-01 05:17:21,2023-03-16 03:10:22,0.094595,73
4,8,9163.48,4581.740000,5932.98,3230.50,2,3230.50,2023-03-05 05:23:19,2023-03-11 02:49:04,0.333333,5
...,...,...,...,...,...,...,...,...,...,...,...
5718,18045,566.98,566.980000,566.98,566.98,1,566.98,2023-03-31 03:21:25,2023-03-31 03:21:25,1.000000,0
5719,18054,740.36,740.360000,740.36,740.36,1,740.36,2023-03-31 04:24:45,2023-03-31 04:24:45,1.000000,0
5720,18060,376.65,376.650000,376.65,376.65,1,376.65,2023-03-31 05:59:52,2023-03-31 05:59:52,1.000000,0
5721,18072,352.49,352.490000,352.49,352.49,1,352.49,2023-03-31 03:17:50,2023-03-31 03:17:50,1.000000,0


In [118]:
labels.describe()

,agent_id,charge_off_within_3_months,charge_off_within_6_months,charge_off_within_9_months,charge_off_within_12_months
count,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000
mean,12658.073177,0.001632,0.009385,0.018362,0.027476
std,4744.594481,0.040370,0.096428,0.134267,0.163476
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11312.750000,0.000000,0.000000,0.000000,0.000000
50%,13616.500000,0.000000,0.000000,0.000000,0.000000
75%,15884.500000,0.000000,0.000000,0.000000,0.000000
max,18190.000000,1.000000,1.000000,1.000000,1.000000


In [119]:
merged_df = labels.merge(aggregated, on='agent_id', how='left')

In [120]:
# Convert timestamp columns to datetime
merged_df['timestamp_min'] = pd.to_datetime(merged_df['timestamp_min'])
merged_df['timestamp_max'] = pd.to_datetime(merged_df['timestamp_max'])

# Calculate the difference in days
merged_df['timestamp_diff'] = (merged_df['timestamp_max'] - merged_df['timestamp_min']).dt.days

# Drop the original timestamp columns
merged_df.drop(columns=['timestamp_min', 'timestamp_max'], inplace=True)

# Display the updated DataFrame
merged_df.head()

,agent_id,charge_off_within_3_months,charge_off_within_6_months,charge_off_within_9_months,charge_off_within_12_months,credit_balance_mean,credit_balance_max,credit_balance_last,credit_utilization_mean,credit_utilization_max,credit_utilization_last,interest_rate_mean,interest_rate_last,current_missed_payments_sum,current_missed_payments_max,current_missed_payments_last,timestamp_diff
0,0,0,0,0,0,3054.533833,4964.111415,2505.991415,0.623747,0.993,0.501,0.209866,0.209866,0,0,0,90
1,3,0,0,0,0,1826.206224,1990.626004,1990.626004,0.927758,0.995,0.995,0.211185,0.211185,0,0,0,90
2,6,0,0,0,0,649.106037,1279.993400,321.083400,0.227275,0.439,0.107,0.186929,0.187204,0,0,0,90
3,7,0,0,0,0,1621.523367,1984.664026,1941.344026,0.850923,0.998,0.971,0.216252,0.216252,0,0,0,90
4,8,0,0,0,0,5748.472329,9973.203428,7835.223428,0.584846,0.997,0.784,0.219701,0.219975,1,1,0,90


In [121]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Prepare features and targets
X = merged_df.drop(columns=['agent_id', 'charge_off_within_3_months', 'charge_off_within_6_months', 
                             'charge_off_within_9_months', 'charge_off_within_12_months'])
y = merged_df[['charge_off_within_3_months', 'charge_off_within_6_months', 
               'charge_off_within_9_months', 'charge_off_within_12_months']]

# Initialize a dictionary to store results for each target
results = {}

# Iterate over each target variable
for target in y.columns:
    print(f"Training model for target: {target}")
    
    # Split data for this target
    X_train, X_test, y_train, y_test = train_test_split(X, y[target], test_size=0.2, random_state=42)
    
    # Convert to DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    # Define parameters for binary classification
    params = {
        'objective': 'binary:logistic',  # Binary classification
        'max_depth': 6,
        'eta': 0.3,
        'eval_metric': 'logloss'
    }
    
    # Train the model
    num_rounds = 100
    bst = xgb.train(params, dtrain, num_rounds)
    
    # Make predictions
    preds = bst.predict(dtest)
    preds_binary = [1 if pred > 0.5 else 0 for pred in preds]  # Convert probabilities to binary predictions
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, preds_binary)
    precision = precision_score(y_test, preds_binary, zero_division=0)
    recall = recall_score(y_test, preds_binary, zero_division=0)
    f1 = f1_score(y_test, preds_binary, zero_division=0)
    cm = confusion_matrix(y_test, preds_binary)
    report = classification_report(y_test, preds_binary, zero_division=0)
    
    # Print metrics
    print(f"Accuracy for {target}: {accuracy:.4f}")
    print(f"Precision for {target}: {precision:.4f}")
    print(f"Recall for {target}: {recall:.4f}")
    print(f"F1-score for {target}: {f1:.4f}")
    print(f"Confusion Matrix for {target}:\n{cm}")
    print(f"Classification Report for {target}:\n{report}")
    
    # Save results and model
    results[target] = {
        'model': bst,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm,
        'classification_report': report
    }
    bst.save_model(f"xgboost_model_{target}.json")

# Summary of results
for target, result in results.items():
    print(f"Model for {target}:")
    print(f"  Accuracy: {result['accuracy']:.4f}")
    print(f"  Precision: {result['precision']:.4f}")
    print(f"  Recall: {result['recall']:.4f}")
    print(f"  F1-score: {result['f1_score']:.4f}")


Training model for target: charge_off_within_3_months
Accuracy for charge_off_within_3_months: 0.9993
Precision for charge_off_within_3_months: 0.0000
Recall for charge_off_within_3_months: 0.0000
F1-score for charge_off_within_3_months: 0.0000
Confusion Matrix for charge_off_within_3_months:
[[1470    0]
 [   1    0]]
Classification Report for charge_off_within_3_months:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1470
           1       0.00      0.00      0.00         1

    accuracy                           1.00      1471
   macro avg       0.50      0.50      0.50      1471
weighted avg       1.00      1.00      1.00      1471

Training model for target: charge_off_within_6_months
Accuracy for charge_off_within_6_months: 0.9925
Precision for charge_off_within_6_months: 0.0000
Recall for charge_off_within_6_months: 0.0000
F1-score for charge_off_within_6_months: 0.0000
Confusion Matrix for charge_off_within_6_months:
[[14

In [122]:
merged_df2 = merged_df.merge(aggregated_payments, on='agent_id', how='left')

In [123]:
merged_df2.isna().sum()

agent_id                           0
charge_off_within_3_months         0
charge_off_within_6_months         0
charge_off_within_9_months         0
charge_off_within_12_months        0
credit_balance_mean                0
credit_balance_max                 0
credit_balance_last                0
credit_utilization_mean            0
credit_utilization_max             0
credit_utilization_last            0
interest_rate_mean                 0
interest_rate_last                 0
current_missed_payments_sum        0
current_missed_payments_max        0
current_missed_payments_last       0
timestamp_diff                     0
amount_sum                      1629
amount_mean                     1629
amount_max                      1629
amount_min                      1629
amount_count                    1629
amount_last                     1629
timestamp_min                   1629
timestamp_max                   1629
payment_frequency               1629
time_between_first_last         1629
d

In [124]:
merged_df2['amount_sum'].fillna(0, inplace=True)
merged_df2['amount_mean'].fillna(0, inplace=True)
merged_df2['amount_max'].fillna(0, inplace=True)
merged_df2['amount_min'].fillna(0, inplace=True)
merged_df2['amount_count'].fillna(0, inplace=True)
merged_df2['amount_last'].fillna(0, inplace=True)
merged_df2['payment_frequency'].fillna(0, inplace=True)
merged_df2.drop(columns=['timestamp_min', 'timestamp_max','time_between_first_last'], inplace=True)


/var/folders/g3/nf_q_x0d0q3_12n_qrkr73m80000gn/T/ipykernel_94613/2776307123.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df2['amount_sum'].fillna(0, inplace=True)
/var/folders/g3/nf_q_x0d0q3_12n_qrkr73m80000gn/T/ipykernel_94613/2776307123.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

In [125]:
merged_df3 = merged_df2.merge(aggregated_transactions, on='agent_id', how='left')


In [126]:
merged_df3.isna().sum()

agent_id                         0
charge_off_within_3_months       0
charge_off_within_6_months       0
charge_off_within_9_months       0
charge_off_within_12_months      0
credit_balance_mean              0
credit_balance_max               0
credit_balance_last              0
credit_utilization_mean          0
credit_utilization_max           0
credit_utilization_last          0
interest_rate_mean               0
interest_rate_last               0
current_missed_payments_sum      0
current_missed_payments_max      0
current_missed_payments_last     0
timestamp_diff                   0
amount_sum_x                     0
amount_mean_x                    0
amount_max_x                     0
amount_min                       0
amount_count_x                   0
amount_last                      0
payment_frequency                0
amount_sum_y                    77
amount_mean_y                   77
amount_max_y                    77
amount_count_y                  77
online_sum          

In [129]:
merged_df3['amount_sum_y'].fillna(0, inplace=True)
merged_df3['amount_mean_y'].fillna(0, inplace=True)
merged_df3['amount_max_y'].fillna(0, inplace=True)
merged_df3['amount_count_y'].fillna(0, inplace=True)
merged_df3['online_sum'].fillna(0, inplace=True)
merged_df3['online_mean'].fillna('0', inplace=True)
merged_df3['Misc'].fillna(0, inplace=True)
merged_df3['Retail'].fillna(0, inplace=True)
merged_df3['Business'].fillna(0, inplace=True)
merged_df3['Clothing'].fillna(0, inplace=True)
merged_df3['Agricultural'].fillna(0, inplace=True)
merged_df3['Contractor'].fillna(0, inplace=True)
merged_df3['Transportation'].fillna(0, inplace=True)
merged_df3['Utility'].fillna(0, inplace=True)
merged_df3['Professional'].fillna(0, inplace=True)
merged_df3['approved_count'].fillna(0, inplace=True)
merged_df3['declined_count'].fillna(0, inplace=True)
merged_df3.drop(columns=['timestamp_min', 'timestamp_max'], inplace=True)


/var/folders/g3/nf_q_x0d0q3_12n_qrkr73m80000gn/T/ipykernel_94613/902087923.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df3['amount_sum_y'].fillna(0, inplace=True)
/var/folders/g3/nf_q_x0d0q3_12n_qrkr73m80000gn/T/ipykernel_94613/902087923.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

In [130]:
merged_final=merged_df3.copy()

In [131]:
merged_final

,agent_id,charge_off_within_3_months,charge_off_within_6_months,charge_off_within_9_months,charge_off_within_12_months,credit_balance_mean,credit_balance_max,credit_balance_last,credit_utilization_mean,credit_utilization_max,...,Retail,Business,Clothing,Agricultural,Contractor,Transportation,Utility,Professional,approved_count,declined_count
0,0,0,0,0,0,3054.533833,4964.111415,2505.991415,0.623747,0.993,...,7.0,5.0,8.0,8.0,7.0,15.0,11.0,7.0,66.0,21.0
1,3,0,0,0,0,1826.206224,1990.626004,1990.626004,0.927758,0.995,...,27.0,53.0,12.0,20.0,5.0,11.0,10.0,16.0,23.0,183.0
2,6,0,0,0,0,649.106037,1279.993400,321.083400,0.227275,0.439,...,8.0,0.0,7.0,4.0,0.0,5.0,2.0,4.0,90.0,0.0
3,7,0,0,0,0,1621.523367,1984.664026,1941.344026,0.850923,0.998,...,17.0,36.0,14.0,16.0,10.0,8.0,10.0,8.0,48.0,112.0
4,8,0,0,0,0,5748.472329,9973.203428,7835.223428,0.584846,0.997,...,9.0,34.0,11.0,8.0,3.0,9.0,6.0,6.0,75.0,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,18185,0,0,0,0,691.140065,691.140065,691.140065,0.346000,0.346,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7348,18186,0,0,0,0,1167.109351,1167.109351,1167.109351,0.389000,0.389,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7349,18187,0,0,0,0,6.503746,6.503746,6.503746,0.145000,0.145,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7350,18189,0,0,0,0,575.152999,575.152999,575.152999,0.288000,0.288,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [132]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Prepare features and targets
X = merged_final_one_hot.drop(columns=['agent_id', 'charge_off_within_3_months', 'charge_off_within_6_months', 
                             'charge_off_within_9_months', 'charge_off_within_12_months'])
y = merged_final_one_hot[['charge_off_within_3_months', 'charge_off_within_6_months', 
               'charge_off_within_9_months', 'charge_off_within_12_months']]

# Initialize a dictionary to store results for each target
results = {}

# Iterate over each target variable
for target in y.columns:
    print(f"Training model for target: {target}")
    
    # Split data for this target
    X_train, X_test, y_train, y_test = train_test_split(X, y[target], test_size=0.2, random_state=42)
    
    # Convert to DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    # Define parameters for binary classification
    params = {
        'objective': 'binary:logistic',  # Binary classification
        'max_depth': 6,
        'eta': 0.3,
        'eval_metric': 'logloss'
    }
    
    # Train the model
    num_rounds = 100
    bst = xgb.train(params, dtrain, num_rounds)
    
    # Make predictions
    preds = bst.predict(dtest)
    preds_binary = [1 if pred > 0.5 else 0 for pred in preds]  # Convert probabilities to binary predictions
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, preds_binary)
    precision = precision_score(y_test, preds_binary, zero_division=0)
    recall = recall_score(y_test, preds_binary, zero_division=0)
    f1 = f1_score(y_test, preds_binary, zero_division=0)
    cm = confusion_matrix(y_test, preds_binary)
    report = classification_report(y_test, preds_binary, zero_division=0)
    
    # Print metrics
    print(f"Accuracy for {target}: {accuracy:.4f}")
    print(f"Precision for {target}: {precision:.4f}")
    print(f"Recall for {target}: {recall:.4f}")
    print(f"F1-score for {target}: {f1:.4f}")
    print(f"Confusion Matrix for {target}:\n{cm}")
    print(f"Classification Report for {target}:\n{report}")
    
    # Save results and model
    results[target] = {
        'model': bst,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm,
        'classification_report': report
    }
    bst.save_model(f"xgboost_model_{target}.json")

# Summary of results
for target, result in results.items():
    print(f"Model for {target}:")
    print(f"  Accuracy: {result['accuracy']:.4f}")
    print(f"  Precision: {result['precision']:.4f}")
    print(f"  Recall: {result['recall']:.4f}")
    print(f"  F1-score: {result['f1_score']:.4f}")


Training model for target: charge_off_within_3_months
Accuracy for charge_off_within_3_months: 0.9993
Precision for charge_off_within_3_months: 0.0000
Recall for charge_off_within_3_months: 0.0000
F1-score for charge_off_within_3_months: 0.0000
Confusion Matrix for charge_off_within_3_months:
[[1470    0]
 [   1    0]]
Classification Report for charge_off_within_3_months:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1470
           1       0.00      0.00      0.00         1

    accuracy                           1.00      1471
   macro avg       0.50      0.50      0.50      1471
weighted avg       1.00      1.00      1.00      1471

Training model for target: charge_off_within_6_months
Accuracy for charge_off_within_6_months: 0.9918
Precision for charge_off_within_6_months: 0.0000
Recall for charge_off_within_6_months: 0.0000
F1-score for charge_off_within_6_months: 0.0000
Confusion Matrix for charge_off_within_6_months:
[[14